In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pickle
import joblib
import csv

In [2]:
# Charger les données
data = pd.read_csv('Données/dataset.csv')
team_dict = pd.read_csv('Données/team_dict.csv')
#match = pd.read_csv('Données/predicted_Chelsea_Arsenal_20240628.csv')

# Remplir les valeurs manquantes avec 0 (si nécessaire)
#data.fillna(0, inplace=True)

# Charger les modèles
random_forest = joblib.load('Modèles/Random_forest.sav')

with open('Modèles/models.pkl', 'rb') as file:
    linear_models = pickle.load(file)

with open('Modèles/scalers.pkl', 'rb') as file:
    linear_scalers = pickle.load(file)

c:\Users\sweez\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sweez\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sweez\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.3.0 when using version 1.5.0. This migh

In [3]:
team_ids = team_dict.set_index('Team')['ID'].to_dict()

In [4]:
# Sélectionner les fonctionnalités d'apprentissage du modèle
features = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC'
            , 'ATGC', 'HTP', 'ATP', 'MW', 'HTFormPts', 'ATFormPts'
            , 'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5'
            , 'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'
            , 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts']

In [7]:
# Saisir l'équipe à domicile et l'équipe à l'extérieur
home_team = input("Enter the Home Team: ")
away_team = input("Enter the Away Team: ")

In [11]:
# Filtrer les données pour les équipes spécifiques
filtered_data = data.loc[np.where((data['HomeTeam'] == home_team) | (data['AwayTeam'] == away_team))] #data.loc[np.where((data['HomeTeam'] == home_team) | (data['AwayTeam'] == away_team))]
filtered_data.shape

(666, 40)

In [12]:
if not filtered_data.empty:

    # Filtrer uniquement les colonnes numériques pour le calcul de la moyenne
    combined_data_numeric = filtered_data[features].mean(axis=0)

    # Créer une DataFrame temporaire pour contenir les valeurs moyennes des caractéristiques / .T transpose la matrice pour obtenir une ligne unique de valeurs moyennes
    combined_data_df = pd.DataFrame(combined_data_numeric).T

    predicted_features = {}
    for feature in features:
        combined_data_scaled = linear_scalers[feature].transform(combined_data_df.drop(columns=[feature]))
        predicted_features[feature] = linear_models[feature].predict(combined_data_scaled)[0]

    # Créer un DataFrame avec les résultats
    results = pd.DataFrame({
        'HomeTeam': [home_team],
        'AwayTeam': [away_team],
        'FTHG': [predicted_features['FTHG']],
        'FTAG': [predicted_features['FTAG']],
        'FTR': [np.nan],  # La colonne FTR est initialisée avec NaN pour être prédite ensuite
        'HTGS': [predicted_features['HTGS']],
        'ATGS': [predicted_features['ATGS']],
        'HTGC': [predicted_features['HTGC']],
        'ATGC': [predicted_features['ATGC']],
        'HTP': [predicted_features['HTP']],
        'ATP': [predicted_features['ATP']],
        'MW': [predicted_features['MW']],
        'HTFormPts': [predicted_features['HTFormPts']],
        'ATFormPts': [predicted_features['ATFormPts']],
        'HTWinStreak3': [predicted_features['HTWinStreak3']],
        'HTWinStreak5': [predicted_features['HTWinStreak5']],
        'HTLossStreak3': [predicted_features['HTLossStreak3']],
        'HTLossStreak5': [predicted_features['HTLossStreak5']],
        'ATWinStreak3': [predicted_features['ATWinStreak3']],
        'ATWinStreak5': [predicted_features['ATWinStreak5']],
        'ATLossStreak3': [predicted_features['ATLossStreak3']],
        'ATLossStreak5': [predicted_features['ATLossStreak5']],
        'HTGD': [predicted_features['HTGD']],
        'ATGD': [predicted_features['ATGD']],
        'DiffPts': [predicted_features['DiffPts']],
        'DiffFormPts': [predicted_features['DiffFormPts']]
    })

    # Enregistrer les résultats dans un fichier CSV avec date et noms d'équipe
    current_date = datetime.now().strftime("%Y%m%d")

    # Spécifier le chemin où vous souhaitez enregistrer le fichier CSV
    output_path = 'Prédictions à prédire'
    file_name = f'predicted_{home_team}_{away_team}_{current_date}.csv'
    file_path = os.path.join(output_path, file_name)

    results.to_csv(file_path, index=False)
    print(f"Les prédictions du match sont stockées dans 'predicted_{home_team}_{away_team}_{current_date}.csv'")
    
else:
    print("Les données pour les équipes spécifiées ne sont pas disponibles dans le dataset.")

Les prédictions du match sont stockées dans 'predicted_Arsenal_Chelsea_20240628.csv'


In [13]:
def encoding_teams(df):
    # Convertir les noms d'équipe en IDs en utilisant le dictionnaire team_ids
    df['HomeTeam'] = df['HomeTeam'].replace(team_ids).astype('int32')
    df['AwayTeam'] = df['AwayTeam'].replace(team_ids).astype('int32')

    return df

In [14]:
# Mettre des ids aux équipes
datatest = encoding_teams(results)

In [15]:
X_test = datatest[['HomeTeam', 'AwayTeam'] + features]
y_test = datatest['FTR']

In [16]:
# Prédire si l'équipe à domicile gagne ou non
predicted_results = random_forest.predict(X_test)

# Afficher le nom des équipes prédites et le résultat de la prédiction
for i, (home_team_id, away_team_id, prediction) in enumerate(zip(X_test['HomeTeam'], X_test['AwayTeam'], predicted_results)):
    home_team_name = list(team_ids.keys())[list(team_ids.values()).index(home_team_id)]
    away_team_name = list(team_ids.keys())[list(team_ids.values()).index(away_team_id)]
    
    if prediction == 1:
        # Afficher le nom des équipes prédites et le résultat de la prédiction
        print(f"Match: {home_team_name} vs {away_team_name}, Prediction: {home_team_name} gagne")

    if prediction == 0:
        # Afficher le nom des équipes prédites et le résultat de la prédiction
        print(f"Match: {home_team_name} vs {away_team_name}, Prediction: {home_team_name} égalise ou perd")

print(predicted_results)

Match: Arsenal vs Chelsea, Prediction: Arsenal gagne
[1]


In [17]:
# Chemin du fichier CSV pour stocker les résultats
output_results_path = 'predicted_match_results_test.csv'

# Ouvrir le fichier en mode 'a' pour ajouter des lignes sans écraser le contenu existant
with open(output_results_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Écrire l'en-tête si le fichier est vide (ou à une autre condition, comme vérifier le contenu)
    if file.tell() == 0:
        writer.writerow(['HomeTeam', 'AwayTeam', 'Prediction'])
    else:
        writer.writerow([home_team, away_team, predicted_results[0]])

In [18]:
teams = ["Man City", "Arsenal", "Liverpool", "Aston Villa", "Tottenham", 
         "Chelsea", "Newcastle", "Man United", "West Ham", "Crystal Palace", 
         "Brighton", "Bournemouth", "Fulham", "Wolves", "Everton", 
         "Brentford", "Nottm Forrest", "Luton Town", "Burnley", "Sheffield United"]

# Liste pour stocker toutes les combinaisons de matchs
match_combinations = []

# Parcours de chaque équipe pour créer les paires de matchs
for i in range(len(teams)):
    home_team = teams[i]
    for j in range(len(teams)):
        if i != j:  # Éviter les matchs d'une équipe contre elle-même
            away_team = teams[j]
            match_combinations.append((home_team, away_team))

# Affichage des résultats
for idx, match in enumerate(match_combinations):
    print(f"Match {idx+1}: {match[0]} vs {match[1]}")

Match 1: Man City vs Arsenal
Match 2: Man City vs Liverpool
Match 3: Man City vs Aston Villa
Match 4: Man City vs Tottenham
Match 5: Man City vs Chelsea
Match 6: Man City vs Newcastle
Match 7: Man City vs Man United
Match 8: Man City vs West Ham
Match 9: Man City vs Crystal Palace
Match 10: Man City vs Brighton
Match 11: Man City vs Bournemouth
Match 12: Man City vs Fulham
Match 13: Man City vs Wolves
Match 14: Man City vs Everton
Match 15: Man City vs Brentford
Match 16: Man City vs Nottm Forrest
Match 17: Man City vs Luton Town
Match 18: Man City vs Burnley
Match 19: Man City vs Sheffield United
Match 20: Arsenal vs Man City
Match 21: Arsenal vs Liverpool
Match 22: Arsenal vs Aston Villa
Match 23: Arsenal vs Tottenham
Match 24: Arsenal vs Chelsea
Match 25: Arsenal vs Newcastle
Match 26: Arsenal vs Man United
Match 27: Arsenal vs West Ham
Match 28: Arsenal vs Crystal Palace
Match 29: Arsenal vs Brighton
Match 30: Arsenal vs Bournemouth
Match 31: Arsenal vs Fulham
Match 32: Arsenal vs 